# Problem Formation

Given a Pattern String as an input, we want to know if it contains dark pattern in it. We use a balanced dataset cotaining all the instances in the Princeton dataset which are all dark patterns, and the instances in the 'normie.csv' file which are labeled as NOT dark patterns. Hence we have a balanced dataset consisting of pattern strings with dark pattern and without park patterns.

Then we use this labeled dataset to build and train supervised machine learning models, and select most suitable ones for our project.

----


In [1]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer

# Bernoulli Naive Bayes (Similar as  MultinomialNB), this classifier is suitable for discrete data. The difference between MultinomialNB and BernoulliNB is that while  MultinomialNB works with occurrence counts, BernoulliNB is designed for binary/boolen features, which means in the case of text classification, word occurrence vectores (rather than word count vectors) may be more suitable to be used to train and use this classifier.
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# Evaluation metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

# joblib is a set of tools to provide lightweight pipelining in Python. It provides utilities for saving and loading Python objects that make use of NumPy data structures, efficiently.
import joblib

import matplotlib.pyplot as plt
# import seaborn as sns

## Data Exploration

---
Import the merged dataset, and explore the dataset.

In [2]:
data = pd.read_csv('enriched_data.csv')

In [3]:
data.head(5)

Pattern String  classification
0  Ends in 07:42:09               0
1  Ends in 07:37:10               0
2  Ends in 02:27:10               0
3  Ends in 04:17:10               0
4  Ends in 01:57:10               0

---
`check the dataset information`

There are 7952 NOT NULL instances of pattern strings in the dataset.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7952 entries, 0 to 7951
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Pattern String  7952 non-null   object
 1   classification  7952 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 124.4+ KB


In [5]:
# check the distribution of the target value --- classification.

print('Distribution of the tags:\n{}'.format(data['classification'].value_counts()))

Distribution of the tags:
1    6897
0    1055
Name: classification, dtype: int64


In [6]:
# Change the label into strings

data['classification'].replace({0:'Dark',1:'Not_Dark'}, inplace = True)

print(data.head(5))

print('\nDistribution of the tags:\n{}'.format(data['classification'].value_counts()))

     Pattern String classification
0  Ends in 07:42:09           Dark
1  Ends in 07:37:10           Dark
2  Ends in 02:27:10           Dark
3  Ends in 04:17:10           Dark
4  Ends in 01:57:10           Dark

Distribution of the tags:
Not_Dark    6897
Dark        1055
Name: classification, dtype: int64


In [7]:
# For later training the model, we should remove the duplicate input to reduce overfitting.

data = data.drop_duplicates(subset="Pattern String")

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7952 entries, 0 to 7951
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Pattern String  7952 non-null   object
 1   classification  7952 non-null   object
dtypes: object(2)
memory usage: 186.4+ KB


In [8]:
print(data.head(5))

print('\nDistribution of the tags:\n{}'.format(data['classification'].value_counts()))

     Pattern String classification
0  Ends in 07:42:09           Dark
1  Ends in 07:37:10           Dark
2  Ends in 02:27:10           Dark
3  Ends in 04:17:10           Dark
4  Ends in 01:57:10           Dark

Distribution of the tags:
Not_Dark    6897
Dark        1055
Name: classification, dtype: int64


---
## Data Preparation

In [9]:
Y = data['classification']
X = data['Pattern String']

---
`Encode the target vales into integers` --- 'classification'

In [10]:
encoder = LabelEncoder()
encoder.fit(Y)
y = encoder.transform(Y)
y.shape

(7952,)

In [12]:
# check the mapping of encoding results (from 0 to 1 representing 'Dark', 'Not Dark')


integer_mapping = {label: encoding for encoding, label in enumerate(encoder.classes_)}
print(integer_mapping)

{'Dark': 0, 'Not_Dark': 1}


In [13]:
# Check the frequency distribution of the training pattern classification with pattern classification names.

(unique, counts) = np.unique(Y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[['Dark' 1055]
 ['Not_Dark' 6897]]


In [14]:
# Check the frequency distribution of the encoded training pattern classification with encoded integers.

(unique, counts) = np.unique(y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[   0 1055]
 [   1 6897]]


---
`Encode the textual features into series of vector of numbers`

In [15]:
# First get the word count vector of the pattern string to encode the pattern string.

tv = TfidfVectorizer()
x = tv.fit_transform(X)


In [16]:
# save the CountVectorizer to disk

joblib.dump(tv, 'presence_TfidfVectorizer.joblib')

['presence_TfidfVectorizer.joblib']

---
# Rough Idea about the effect of different classifiers
---

In [17]:
# Five models are tested:
# -- Logistic Regression
# -- Linear Support Vector Machine
# -- Random Forest
# -- Multinomial Naive Bayes
# -- Bernoulli Naive Bayes
# -- KNN

classifiers = [LogisticRegression(), LinearSVC(), RandomForestClassifier(), MultinomialNB(), BernoulliNB(), KNeighborsClassifier()]

In [18]:
# Calculate the accuracies of different classifiers using default settings.

acc = []
pre = []
cm = []

for clf in classifiers:
    y_pred = cross_val_predict(clf, x, y, cv=5, n_jobs = -1)
    acc.append(metrics.accuracy_score(y, y_pred))
    pre.append(metrics.precision_score(y,y_pred, pos_label=0))
    cm.append(metrics.confusion_matrix(y, y_pred))

In [19]:
# List the accuracies of different classifiers.

for i in range(len(classifiers)):
    print("{} accuracy: {:.3f}".format(classifiers[i],acc[i]))
    print("{} precision: {:.3f}".format(classifiers[i],pre[i]))
    print("Confusion Matrix: {}".format(cm[i]))

LogisticRegression() accuracy: 0.943
LogisticRegression() precision: 0.981
Confusion Matrix: [[ 613  442]
 [  12 6885]]
LinearSVC() accuracy: 0.960
LinearSVC() precision: 0.952
Confusion Matrix: [[ 774  281]
 [  39 6858]]
RandomForestClassifier() accuracy: 0.964
RandomForestClassifier() precision: 0.969
Confusion Matrix: [[ 794  261]
 [  25 6872]]
MultinomialNB() accuracy: 0.949
MultinomialNB() precision: 0.930
Confusion Matrix: [[ 702  353]
 [  53 6844]]
BernoulliNB() accuracy: 0.927
BernoulliNB() precision: 0.726
Confusion Matrix: [[ 762  293]
 [ 287 6610]]
KNeighborsClassifier() accuracy: 0.917
KNeighborsClassifier() precision: 0.864
Confusion Matrix: [[ 470  585]
 [  74 6823]]


---
# Bernoulli Naive Bayes Classifier


---
### `Use default setting of classifier hyperparameters`

In [20]:
clf_bnb = BernoulliNB()

In [21]:
y_pred = cross_val_predict(clf_bnb, x, y, cv=5, n_jobs = -1)

In [22]:
clf_bnb.get_params()

{'alpha': 1.0, 'binarize': 0.0, 'class_prior': None, 'fit_prior': True}

---
`use the default setting of hyperparameters of the Bernoulli Naive Bayes classifier`

In [23]:
print("Accuracy:", metrics.accuracy_score(y, y_pred))
print("Precision:", metrics.precision_score(y,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y, y_pred))

Accuracy: 0.9270623742454729
Precision: 0.7264061010486177
Confusion Matrix:
 [[ 762  293]
 [ 287 6610]]


In [24]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0, 1049],
       [   1, 6903]])

---
### `Parameter Tunning of BernoulliNB classifier`
`Define the combination of parameters to be considered`

In [25]:
param_grid = {'alpha':[0,1], 
              'fit_prior':[True, False]}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [26]:
gs = GridSearchCV(clf_bnb,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [27]:
best_bnb = gs.fit(x,y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.5s finished


In [28]:
scores_df = pd.DataFrame(best_bnb.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_alpha', 'param_fit_prior']]

rank_test_score  mean_test_score param_alpha param_fit_prior
0                1         0.927061           1            True
1                2         0.922912           0            True
2                3         0.918509           1           False
3                4         0.878896           0           False

In [29]:
best_bnb.best_params_

{'alpha': 1, 'fit_prior': True}

---
`Save the best BernoulliNB model for future use`

In [30]:
# save the model to local disk

joblib.dump(best_bnb, 'bnb_presence_classifier.joblib')

['bnb_presence_classifier.joblib']

---
# Random Forest Classifier


---
### `Use default setting of classifier hyperparameters`

In [53]:
clf_rf = RandomForestClassifier()

In [54]:
y_pred = cross_val_predict(clf_rf, x, y, cv=5, n_jobs = -1)

In [55]:
clf_rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

---
`use the default setting of hyperparameters of the Random Forest classifier.`

In [56]:
print("Accuracy:", metrics.accuracy_score(y, y_pred))
print("Precision:", metrics.precision_score(y,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y, y_pred))

Accuracy: 0.9629024144869215
Precision: 0.9656862745098039
Confusion Matrix:
 [[ 788  267]
 [  28 6869]]


In [57]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,  816],
       [   1, 7136]])

---
### `Parameter Tunning of Random Forest classifier`
`Define the combination of parameters to be considered`

In [58]:
param_grid = {'bootstrap':[True,False], 
              'criterion':['gini','entropy'],
              'max_depth':[10,20,30,40,50,60,70,80,90,100, None],
              'min_samples_leaf':[1,2,4],
              'min_samples_split':[2,5,10],
              'n_estimators':[100,200,300,400,500,600]}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [59]:
gs = GridSearchCV(clf_rf,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [60]:
best_rf = gs.fit(x,y)

Fitting 5 folds for each of 2376 candidates, totalling 11880 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed: 32.4min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed: 52.7min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed: 76.6min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 88.0min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 112.0min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed: 147.5min
[Parallel(n_jobs=-1)]: Done 7176 tasks      | elapsed: 171.5min
[Parallel(n_jobs=-1)]: Done 8426 tasks      | elapsed: 233.7min
[Parallel(n_jobs=-1)]: Done 9776 tasks      | elapsed: 290.8min
[Parallel(n_jobs=-1)]: Done 11226 tasks 

In [61]:
scores_df = pd.DataFrame(best_rf.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_bootstrap', 'param_criterion','param_max_depth','param_min_samples_leaf','param_min_samples_split','param_n_estimators']]

rank_test_score  mean_test_score param_bootstrap param_criterion  \
0                   1         0.964665           False            gini   
1                   2         0.964664           False         entropy   
2                   2         0.964664           False         entropy   
3                   4         0.964664           False         entropy   
4                   5         0.964540            True            gini   
...               ...              ...             ...             ...   
2371             2372         0.867832           False            gini   
2372             2372         0.867832            True            gini   
2373             2374         0.867832           False         entropy   
2374             2375         0.867706            True            gini   
2375             2375         0.867706            True            gini   

     param_max_depth param_min_samples_leaf param_min_samples_split  \
0               None                      1                       5   
1               None                      1                       2   
2                100                      1                       5   
3               None                      1                      10   
4                 80                      1                       2   
...              ...                    ...                     ...   
2371              10                      2                       5   
2372              10                      2                      10   
2373              10                      1                      10   
2374              10                      4                       5   
2375              10                      4                      10   

     param_n_estimators  
0                   300  
1                   600  
2                   200  
3                   500  
4                   100  
...                 ...  
2371                500  
2372                500  
2373                300  
2374                100  
2375                600  

[2376 rows x 8 columns]

In [62]:
best_rf.best_params_

{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 300}

---
`Save the best Random Forest model for future use`

In [63]:
# save the model to local disk

joblib.dump(best_rf, 'rf_presence_classifier.joblib')

['rf_presence_classifier.joblib']

---
# SVM Classifier


---
### `Use default setting of classifier hyperparameters`

In [31]:
clf_svm = LinearSVC()

In [32]:
y_pred = cross_val_predict(clf_svm, x, y, cv=5, n_jobs = -1)

In [33]:
clf_svm.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

---
`use the default setting of hyperparameters of the Random Forest classifier.`

In [34]:
print("Accuracy:", metrics.accuracy_score(y, y_pred))
print("Precision:", metrics.precision_score(y,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y, y_pred))

Accuracy: 0.959758551307847
Precision: 0.9520295202952029
Confusion Matrix:
 [[ 774  281]
 [  39 6858]]


In [35]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,  813],
       [   1, 7139]])

---
### `Parameter Tunning of SVM classifier`
`Define the combination of parameters to be considered`

In [36]:
param_grid = {'C':[0.1,1,10,100],
              'penalty':['l1','l2']}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [37]:
gs = GridSearchCV(clf_svm,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [38]:
best_svm = gs.fit(x,y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.6s finished


In [39]:
scores_df = pd.DataFrame(best_svm.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_penalty', 'param_C']]

rank_test_score  mean_test_score param_penalty param_C
0                1         0.959760            l2       1
1                2         0.956113            l2      10
2                3         0.952718            l2     100
3                4         0.949323            l2     0.1
4                5              NaN            l1     0.1
5                6              NaN            l1       1
6                7              NaN            l1      10
7                8              NaN            l1     100

In [40]:
best_svm.best_params_

{'C': 1, 'penalty': 'l2'}

---
`Save the best SVM model for future use`

In [41]:
# save the model to local disk

joblib.dump(best_svm, 'svm_presence_classifier.joblib')

['svm_presence_classifier.joblib']

---
# Logistic Regression Classifier


---
### `Use default setting of classifier hyperparameters`

In [42]:
clf_lr = LogisticRegression()

In [43]:
y_pred = cross_val_predict(clf_lr, x, y, cv=5, n_jobs = -1)

In [44]:
clf_lr.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

---
`use the default setting of hyperparameters of the Random Forest classifier.`

In [45]:
print("Accuracy:", metrics.accuracy_score(y, y_pred))
print("Precision:", metrics.precision_score(y,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y, y_pred))

Accuracy: 0.942907444668008
Precision: 0.9808
Confusion Matrix:
 [[ 613  442]
 [  12 6885]]


In [46]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,  625],
       [   1, 7327]])

---
### `Parameter Tunning of Logistic Regression classifier`
`Define the combination of parameters to be considered`

In [47]:
param_grid = {'penalty':['l1','l2'], 
              'solver':['lbfgs','newton-cg','sag']}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [48]:
gs = GridSearchCV(clf_lr,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [49]:
best_lr = gs.fit(x,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.6s finished


In [50]:
scores_df = pd.DataFrame(best_lr.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score','param_penalty', 'param_solver']]

rank_test_score  mean_test_score param_penalty param_solver
0                1         0.942911            l2        lbfgs
1                1         0.942911            l2    newton-cg
2                1         0.942911            l2          sag
3                4              NaN            l1        lbfgs
4                5              NaN            l1    newton-cg
5                6              NaN            l1          sag

In [51]:
best_lr.best_params_

{'penalty': 'l2', 'solver': 'lbfgs'}

---
`Save the best SVM model for future use`

In [52]:
# save the model to local disk

joblib.dump(best_lr, 'lr_presence_classifier.joblib')

['lr_presence_classifier.joblib']